# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [ ]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)
# Track the sum
sum = 0
for sqrt_value in generator:
    sum += sqrt_value
    print(sqrt_value)
print("Sum",sum)


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [ ]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

# Solutions

You can use these solutions to self check your results, or to check how the answer can be obtained if you get stuck.

## Solution 1

In [ ]:
def sum_of_generator_outputs(generator, limit):
    return sum(next(generator) for _ in range(limit))

# Example usage:
limit_1 = 5
generator_1 = square_root_generator(limit_1)
result_1 = sum_of_generator_outputs(generator_1, limit_1)
print(f"The sum of the outputs for limit={limit_1} is: {result_1}")


def nth_yielded_number(generator, n):
    for _ in range(n - 1):
        next(generator)
    return next(generator)

# Example usage:
n = 13
generator_2 = square_root_generator(n)
result_2 = nth_yielded_number(generator_2, n)
print(f"The {n}th number yielded is: {result_2}")


## Solution 2: Append a generator

Load your first generator first, and then load the second one using the "append" operation. Since they have overlapping IDs, some records will appear multiple times.

After loading, you should have a total of 11 records.

Question: Calculate the sum of ages of all the people loaded as described above

In [1]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [2]:
# Importing the DLT library
import dlt

# Create a DLT pipeline for the first generator `people_1`
# The pipeline is set to load data into a DuckDB database with the dataset named 'people'
people_1_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')

# Run the pipeline for the first generator, creating or replacing the table 'people'
info = people_1_pipeline.run(people_1(),
                             table_name="people",
                             write_disposition="replace")

print(f"{info}\n\n")


# Create a second DLT pipeline for the generator `people_2`, targeting the same DuckDB database and dataset
people_2_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')

# Run the second pipeline, appending data from `people_2` to the existing 'people' table
info = people_2_pipeline.run(people_2(),
                             table_name="people",
                             write_disposition="append")

print(f"{info}\n\n")


# Importing the DuckDB library
import duckdb

# Connect to the DuckDB database created by the first generator
conn = duckdb.connect(f"{people_1_pipeline.pipeline_name}.duckdb")

# Setting the search path to the dataset 'people' and displaying available tables
conn.sql(f"SET search_path = '{people_1_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))


# Fetching the appended data from the 'people' table and displaying it
data = conn.sql("SELECT * FROM people").df()
display(data)

# Calculate the sum of ages from the combined data of `people_1` and `people_2` in the 'people' table
sum_of_ages_p1_p2 = conn.execute("SELECT SUM(age) FROM people").fetchone()[0]
print(f"\n\nSum of ages from generators `people_1()` and `people_2()` combined: {sum_of_ages_p1_p2}")


Pipeline dlt_ipykernel_launcher load step completed in 0.48 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:///E:\Dev\Data Engineering\Data Engineering Bootcamp\Current\my-data-engineering-work\workshops\dlt_resources\dlt_ipykernel_launcher.duckdb location to store data
Load package 1709155884.4942026 is LOADED and contains no failed jobs


Pipeline dlt_ipykernel_launcher load step completed in 0.44 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:///E:\Dev\Data Engineering\Data Engineering Bootcamp\Current\my-data-engineering-work\workshops\dlt_resources\dlt_ipykernel_launcher.duckdb location to store data
Load package 1709155885.572878 is LOADED and contains no failed jobs


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1709155884.4942026,IlhDMsXXOL+BlA,None
1,2,Person_2,27,City_A,1709155884.4942026,xPwMKS37MvXY1A,None
2,3,Person_3,28,City_A,1709155884.4942026,EFkuoC9NBCENjA,None
3,4,Person_4,29,City_A,1709155884.4942026,kQCZwFqBaU2fXw,None
4,5,Person_5,30,City_A,1709155884.4942026,G4Zhm1TqdikYqw,None
5,3,Person_3,33,City_B,1709155885.572878,MsC6T91aOIRrFA,Job_3
6,4,Person_4,34,City_B,1709155885.572878,fyGe9yQVusGT0A,Job_4
7,5,Person_5,35,City_B,1709155885.572878,hznuiYSd18l3pg,Job_5
8,6,Person_6,36,City_B,1709155885.572878,za73MTsDAoUz7A,Job_6
9,7,Person_7,37,City_B,1709155885.572878,nW/EzktOabeIRg,Job_7




Sum of ages from generators `people_1()` and `people_2()` combined: 353


## Solution 3: Merge a generator

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

In [3]:
import dlt

# Set up a DLT pipeline.
# Currently using DuckDB for local testing, but it can be switched to BigQuery for production.
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people_merged')

# Load data from the first generator `people_1` into 'people_merge' table.
# This operation will replace any existing data in the table.
# A primary key 'ID' is specified for potential future merge operations.
info = generators_pipeline.run(people_1(),
                               table_name="people_merged",
                               write_disposition="replace",
                               primary_key="ID")

# Print metadata of the loading process for the first generator.
print(f"{info}\n\n")

# Load data from the second generator `people_2` into the same 'people_merge' table.
# This operation will merge the new data with existing data based on the primary key 'ID'.
info = generators_pipeline.run(people_2(),
                               table_name="people_merged",
                               write_disposition="merge",
                               primary_key="ID")

# Print metadata of the loading process for the second generator.
print(f"{info}\n\n")

import duckdb

# Establish a connection to the DuckDB database created by the pipeline.
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# Set the search path to the dataset 'people_merge' and display the available tables.
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# Display the merged data from the 'people_merged' table.
print("\n\n\nData from the 'people_merged' table:")
data = conn.sql("SELECT * FROM people_merged").df()
display(data)

# Calculate and display the sum of ages from the merged data in 'people_merged' table.
sum_of_ages_p1_p2 = conn.execute("SELECT SUM(age) FROM people_merged").fetchone()[0]
print(f"\n\nSum of ages of people in generator `people_1()` merged with generator `people_2()` is: {sum_of_ages_p1_p2}")


Pipeline dlt_ipykernel_launcher load step completed in 0.30 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_merged
The duckdb destination used duckdb:///E:\Dev\Data Engineering\Data Engineering Bootcamp\Current\my-data-engineering-work\workshops\dlt_resources\dlt_ipykernel_launcher.duckdb location to store data
Load package 1709155916.7760425 is LOADED and contains no failed jobs


Pipeline dlt_ipykernel_launcher load step completed in 0.47 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_merged
The duckdb destination used duckdb:///E:\Dev\Data Engineering\Data Engineering Bootcamp\Current\my-data-engineering-work\workshops\dlt_resources\dlt_ipykernel_launcher.duckdb location to store data
Load package 1709155917.5953708 is LOADED and contains no failed jobs


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_merged       │
└─────────────────────┘




Data from the 'people_merged' table:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1709155916.7760425,uh2WgDSTL6S9Kw,None
1,2,Person_2,27,City_A,1709155916.7760425,DACyS+l1xwDamw,None
2,7,Person_7,37,City_B,1709155917.5953708,CJIfTuXFK8G4mw,Job_7
3,8,Person_8,38,City_B,1709155917.5953708,zCmKnCOzZDg1uQ,Job_8
4,4,Person_4,34,City_B,1709155917.5953708,rti9h2c858jaqw,Job_4
5,3,Person_3,33,City_B,1709155917.5953708,NCHulS1j/lsn8w,Job_3
6,6,Person_6,36,City_B,1709155917.5953708,6OipNwK9G7WBiw,Job_6
7,5,Person_5,35,City_B,1709155917.5953708,/9oJXYSMJBGd5w,Job_5




Sum of ages of people in generator `people_1()` merged with generator `people_2()` is: 266
